# Braki danych

Braki danych to poważny problem w analizie danych. Strategie radzenia sobie z brakami danych:

1. Usuwanie 
2. Imputacja

Zdecydowanie się na drugi scenariusz związane jest z wyborem sposobu imputacji danych. Do najprostszych należy zastępowanie braków danych miarami tendencji centralnej:

- zmienna ilościowa - średnia lub mediana,
- zmienna jakościowa - dominanta.

In [1]:
libname bib "/folders/myfolders/przetwarzanie-danych-SAS/";

SAS Connection established. Subprocess id is 5609



Generowanie braków danych

In [2]:
data bib.bank_braki;
set bib.bank;
random=ranuni(123);
random2=ranuni(234);
random3=ranuni(345);
if random < 0.1 then saldo = .;
if random2 < 0.12 then czas = .;
if random3 < 0.2 then pozyczka = ""; 
drop random random2 random3;
run;

Brakujące dane dla zmiennej `saldo` można uzupełnić średnią: 

In [8]:
proc means data=bib.bank_braki n nmiss mean;
var saldo;
run;

In [10]:
data bib.bank_imputacja;
set bib.bank_braki;
if saldo=. then saldo=1366.06;
run;

In [11]:
proc means data=bib.bank_imputacja n nmiss mean;
var saldo;
run;

Nie jest to jednak zbyt eleganckie rozwiązanie ponieważ nasz kod nie jest uniwersalny - wprowadziliśmy magiczną liczbę, która nie wiadomo skąd się wzięła i w przypadku modyfikacji zbioru danych będziemy musieli pamiętać o zmianie także tej wartości.

Dużo lepiej skorzystać z makrozmiennej tworzonej na podstawie danych:

```
data _null_;
set zbior;
call symputx('makrozmienna',zmienna);
run;
```

In [12]:
proc means data=bib.bank_braki n nmiss mean;
var saldo;
output out=srednie_saldo mean=;
run;

In [14]:
data _null_;
set srednie_saldo;
call symputx('sr_saldo',saldo);
run;

%put ***** &sr_saldo;

In [21]:
data bib.bank_imputacja;
set bib.bank_braki;
if saldo=. then saldo=&sr_saldo;
run;

proc means data=bib.bank_imputacja n nmiss mean;
var saldo;
run;

Z kolei braki danych w zmiennej jakościowej np. dla cechy wykształcenie można uzupełnić wartością najczęściej występującą.

In [22]:
proc freq data=bib.bank_braki;
table wykszt / missing out=wyk_freq; 
run;

proc sort data=wyk_freq;
by descending count;
run;

In [23]:
data _null_;
set wyk_freq(obs=1);
call symputx('wyk_max',wykszt);
run;

%put ***** &wyk_max;

In [25]:
data bib.bank_imputacja;
set bib.bank_braki;
if wykszt="" then wykszt="&wyk_max";
run;

proc freq data=bib.bank_imputacja;
table wykszt / missing; 
run;